In [2]:
!pip install tensorflow torch torchvision transformers pandas opencv-python matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
import os
import pandas as pd
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

# Paths
IMAGE_FOLDER = "/content/drive/MyDrive/subset_data/Subset_Images/Images"
CSV_FILE = "/content/drive/MyDrive/subset_data/Filtered_Subset.csv"

# Load the filtered CSV
df = pd.read_csv(CSV_FILE)

# Define image preprocessing steps
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for ResNet
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for ResNet
])

# Load ResNet-50 (pretrained on ImageNet)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model = model.to(device)
model.eval()  # Set to evaluation mode

# Remove the classification layer to extract features
model = torch.nn.Sequential(*list(model.children())[:-1])

# Function to extract features
def extract_features(image_path):
    try:
        image = Image.open(image_path).convert("RGB")  # Open and convert to RGB
        image = transform(image).unsqueeze(0).to(device)  # Preprocess and add batch dimension
        with torch.no_grad():
            features = model(image).squeeze().cpu().numpy()  # Extract features
        return features
    except Exception as e:
        print(f"❌ Error processing {image_path}: {e}")
        return None

# Extract features for all images
features_dict = {}
for i, row in df.iterrows():
    image_id = row["image_id"]
    image_path = os.path.join(IMAGE_FOLDER, image_id)

    if os.path.exists(image_path):
        features = extract_features(image_path)
        if features is not None:
            features_dict[image_id] = features
    else:
        print(f"❌ Missing Image: {image_id}")

# Print some sample features
for img_id, feat in list(features_dict.items())[:5]:  # Print first 5 feature vectors
    print(f"📸 {img_id}: Feature Shape = {feat.shape}")

print(f"✅ Feature extraction complete! Extracted features for {len(features_dict)} images.")


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 146MB/s]


📸 000001.jpg: Feature Shape = (2048,)
📸 000008.jpg: Feature Shape = (2048,)
📸 000009.jpg: Feature Shape = (2048,)
📸 000010.jpg: Feature Shape = (2048,)
📸 000011.jpg: Feature Shape = (2048,)
✅ Feature extraction complete! Extracted features for 1512 images.


In [2]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Hyperparameters
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.001
OPTIMIZERS = {
    "Adam": optim.Adam,
    "SGD": optim.SGD,
    "AdamW": optim.AdamW,
    "RMSprop": optim.RMSprop
}

# Load Dataset
df = pd.read_csv(CSV_FILE)
df["image_id"] = df["image_id"].str.strip()  # Remove unwanted spaces

# 🔹 Fix: Assign Labels if Missing
if "label" not in df.columns:
    print("⚠️ 'label' column not found! Assigning labels automatically...")
    df["label"] = pd.factorize(df["text_description"])[0]  # Convert text categories to numbers

print("✅ Unique Labels:", df["label"].unique())  # Debugging step

# Custom Dataset
class ImageTextDataset(Dataset):
    def __init__(self, df, image_folder, transform=None):
        self.df = df
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_folder, row["image_id"])
        image = Image.open(image_path).convert("RGB")
        label = int(row["label"])  # 🔹 Fix: Ensure labels are integers

        if self.transform:
            image = self.transform(image)

        return image, label

# Image Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Prepare DataLoader
dataset = ImageTextDataset(df, IMAGE_FOLDER, transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Define Model (ResNet50 for feature extraction)
class ImageClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ImageClassifier, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Function to Train and Evaluate Model
def train_and_evaluate(optimizer_name):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ImageClassifier(num_classes=len(df["label"].unique())).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = OPTIMIZERS[optimizer_name](model.parameters(), lr=LEARNING_RATE)

    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        all_preds, all_labels = [], []

        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        # Calculate Metrics
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
        f1 = f1_score(all_labels, all_preds, average='weighted')

        print(f"Optimizer: {optimizer_name} | Epoch {epoch+1}/{EPOCHS} | "
              f"Loss: {running_loss:.4f} | Acc: {accuracy:.4f} | "
              f"Precision: {precision:.4f} | Recall: {recall:.4f} | F1-score: {f1:.4f}")

    return {"optimizer": optimizer_name, "loss": running_loss, "accuracy": accuracy,
            "precision": precision, "recall": recall, "f1_score": f1}

# Run Experiments with Different Optimizers
results = []
for opt in OPTIMIZERS:
    print(f"\n🔹 Training with {opt} optimizer...")
    results.append(train_and_evaluate(opt))

# Display Results
print("\nFinal Results:")
for res in results:
    print(res)

⚠️ 'label' column not found! Assigning labels automatically...
✅ Unique Labels: [   0    1    2 ... 1506 1507 1508]

🔹 Training with Adam optimizer...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Optimizer: Adam | Epoch 1/10 | Loss: 726.4974 | Acc: 0.0000 | Precision: 0.9722 | Recall: 0.0000 | F1-score: 0.0000
Optimizer: Adam | Epoch 2/10 | Loss: 691.8957 | Acc: 0.0000 | Precision: 0.9894 | Recall: 0.0000 | F1-score: 0.0000
Optimizer: Adam | Epoch 3/10 | Loss: 682.0803 | Acc: 0.0000 | Precision: 0.9868 | Recall: 0.0000 | F1-score: 0.0000
Optimizer: Adam | Epoch 4/10 | Loss: 674.0501 | Acc: 0.0000 | Precision: 0.9854 | Recall: 0.0000 | F1-score: 0.0000
Optimizer: Adam | Epoch 5/10 | Loss: 669.0113 | Acc: 0.0007 | Precision: 0.9881 | Recall: 0.0007 | F1-score: 0.0000
Optimizer: Adam | Epoch 6/10 | Loss: 666.4567 | Acc: 0.0007 | Precision: 0.9821 | Recall: 0.0007 | F1-score: 0.0000
Optimizer: Adam | Epoch 7/10 | Loss: 665.9752 | Acc: 0.0007 | Precision: 0.9762 | Recall: 0.0007 | F1-score: 0.0000
Optimizer: Adam | Epoch 8/10 | Loss: 659.6860 | Acc: 0.0013 | Precision: 0.9623 | Recall: 0.0013 | F1-score: 0.0000
Optimizer: Adam | Epoch 9/10 | Loss: 648.7975 | Acc: 0.0007 | Precision:

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Optimizer: SGD | Epoch 1/10 | Loss: 701.3290 | Acc: 0.0007 | Precision: 0.9259 | Recall: 0.0007 | F1-score: 0.0000
Optimizer: SGD | Epoch 2/10 | Loss: 698.3430 | Acc: 0.0007 | Precision: 0.9187 | Recall: 0.0007 | F1-score: 0.0000
Optimizer: SGD | Epoch 3/10 | Loss: 695.8061 | Acc: 0.0007 | Precision: 0.9187 | Recall: 0.0007 | F1-score: 0.0000
Optimizer: SGD | Epoch 4/10 | Loss: 692.8532 | Acc: 0.0020 | Precision: 0.9068 | Recall: 0.0020 | F1-score: 0.0001
Optimizer: SGD | Epoch 5/10 | Loss: 690.1270 | Acc: 0.0033 | Precision: 0.9032 | Recall: 0.0033 | F1-score: 0.0005
Optimizer: SGD | Epoch 6/10 | Loss: 686.9966 | Acc: 0.0026 | Precision: 0.8897 | Recall: 0.0026 | F1-score: 0.0003
Optimizer: SGD | Epoch 7/10 | Loss: 684.1441 | Acc: 0.0040 | Precision: 0.8924 | Recall: 0.0040 | F1-score: 0.0004
Optimizer: SGD | Epoch 8/10 | Loss: 681.6577 | Acc: 0.0079 | Precision: 0.8833 | Recall: 0.0079 | F1-score: 0.0023
Optimizer: SGD | Epoch 9/10 | Loss: 678.5424 | Acc: 0.0079 | Precision: 0.8744 |

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Optimizer: AdamW | Epoch 1/10 | Loss: 737.9503 | Acc: 0.0000 | Precision: 0.9683 | Recall: 0.0000 | F1-score: 0.0000
Optimizer: AdamW | Epoch 2/10 | Loss: 698.0061 | Acc: 0.0007 | Precision: 0.9894 | Recall: 0.0007 | F1-score: 0.0000
Optimizer: AdamW | Epoch 3/10 | Loss: 696.8061 | Acc: 0.0007 | Precision: 0.9927 | Recall: 0.0007 | F1-score: 0.0000
Optimizer: AdamW | Epoch 4/10 | Loss: 697.5798 | Acc: 0.0013 | Precision: 0.9980 | Recall: 0.0013 | F1-score: 0.0000
Optimizer: AdamW | Epoch 5/10 | Loss: 695.4748 | Acc: 0.0007 | Precision: 0.9980 | Recall: 0.0007 | F1-score: 0.0000
Optimizer: AdamW | Epoch 6/10 | Loss: 692.6504 | Acc: 0.0013 | Precision: 0.9980 | Recall: 0.0013 | F1-score: 0.0000
Optimizer: AdamW | Epoch 7/10 | Loss: 685.5666 | Acc: 0.0007 | Precision: 0.9908 | Recall: 0.0007 | F1-score: 0.0000
Optimizer: AdamW | Epoch 8/10 | Loss: 673.1622 | Acc: 0.0013 | Precision: 0.9868 | Recall: 0.0013 | F1-score: 0.0000
Optimizer: AdamW | Epoch 9/10 | Loss: 663.2374 | Acc: 0.0020 | P

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Optimizer: RMSprop | Epoch 1/10 | Loss: 729.1675 | Acc: 0.0000 | Precision: 0.9835 | Recall: 0.0000 | F1-score: 0.0000
Optimizer: RMSprop | Epoch 2/10 | Loss: 697.1846 | Acc: 0.0000 | Precision: 0.9894 | Recall: 0.0000 | F1-score: 0.0000
Optimizer: RMSprop | Epoch 3/10 | Loss: 696.7081 | Acc: 0.0000 | Precision: 0.9921 | Recall: 0.0000 | F1-score: 0.0000
Optimizer: RMSprop | Epoch 4/10 | Loss: 696.5991 | Acc: 0.0007 | Precision: 0.9940 | Recall: 0.0007 | F1-score: 0.0000
Optimizer: RMSprop | Epoch 5/10 | Loss: 696.1553 | Acc: 0.0000 | Precision: 0.9921 | Recall: 0.0000 | F1-score: 0.0000
Optimizer: RMSprop | Epoch 6/10 | Loss: 695.4706 | Acc: 0.0007 | Precision: 0.9881 | Recall: 0.0007 | F1-score: 0.0000
Optimizer: RMSprop | Epoch 7/10 | Loss: 693.7702 | Acc: 0.0000 | Precision: 0.9894 | Recall: 0.0000 | F1-score: 0.0000
Optimizer: RMSprop | Epoch 8/10 | Loss: 686.4743 | Acc: 0.0007 | Precision: 0.9835 | Recall: 0.0007 | F1-score: 0.0000
Optimizer: RMSprop | Epoch 9/10 | Loss: 675.4372